# Demonstration View

View a demonstration by sliding through the frames.
This also plots the z height.

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

folder_format = "lukas"

#recording = "stack_recordings/episode_118"
#recording = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/stacking_sim"
recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/shape_insert", 10

if folder_format == 'max':
    state_recording_fn = "./{}/episode_1.npz".format(recording)
    state_recording = np.load(state_recording_fn)
    ee_positions = state_recording["ee_positions"]
    flow_recording_fn = "./{}/episode_1_img.npz".format(recording)
    flow_recording = np.load(flow_recording_fn)["img"]
else:
    state_recording_fn = "{}/episode_{}.npz".format(recording, episode_num)
    flow_recording_fn = "{}/episode_{}.npz".format(recording, episode_num)
    
    state_recording = np.load(state_recording_fn)["robot_state_full"]
    actions = np.load(state_recording_fn)["actions"]
    ee_positions = state_recording[:,:3]
    print(state_recording.shape)
    
    flow_recording = np.load(flow_recording_fn)["rgb_unscaled"]
    try:
        seg_masks = np.load(flow_recording_fn)["seg_masks"]
    except KeyError:
        seg_masks = None
num_frames = flow_recording.shape[0]-1


x = np.linspace(0, 2 * np.pi)
fig, (ax,ax2) = plt.subplots(2,1)
line = ax.imshow(flow_recording[0])
ax.set_axis_off()

#ax2.plot(ee_positions[:,0],label="x")
#ax2.plot(ee_positions[:,1],label="y")
# ax2.plot(ee_positions[:,2],label="z")
# gr_positions = (state_recording[:, -2] > 0.04).astype('float')
gr_positions = actions[:, -1]
ax2.plot(gr_positions,label="gripper")
ax2.set_ylabel("z position")
ax2.set_xlabel("frame number")
vline = ax2.axvline(x=2,color="k")
ax2.legend()

def update(w):
    vline.set_data([w,w],[0,1])
    line.set_data(flow_recording[w])
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0,max=num_frames,step=1,value=0,
                             layout=Layout(width='70%'))
interact(update,w=slider_w)

In [ ]:
#recording = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/stacking"
#n = 0
#keep_frames = (100, 101,138, 170, 185)

keep_frames = (80, 81, 105, 110, 120,  204, 206)


keep_frames = np.array(keep_frames)
print("1 is open, -1 is closed")
print("gripper positions", gr_positions[keep_frames])

keep_array = np.zeros(gr_positions.shape, dtype=bool)
keep_array[keep_frames] = True
print(np.sum(keep_array))

keep_fn = flow_recording_fn.replace(".npz","_keep.npz")
np.savez(keep_fn, keep=keep_array)
print("Saved to",keep_fn)

# Mask View

Mask out the foreground object so that foreground specific flow can be calculated.

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(flow_recording[25])
ax.set_axis_off()

def get_mask(frame, threshold, color_index=0, i=None):
    if seg_masks is not None:
        mask = (seg_masks[i] == 2) if color_index == 0 else (seg_masks[i] == 3)
    else:
        image = frame.copy()
        tmp = image[:,:,color_index] / np.linalg.norm(image,axis=2) 
        mask = tmp > threshold
        #mask = np.logical_not(mask) for yellow
        #print("threshold", threshold)
    return mask

def update(w,i):
    vline.set_data([w,w],[0,1])
    image = flow_recording[i].copy()
    mask = get_mask(image, w/100, i=i)
    print(image.shape)
    print(np.logical_not(mask).shape)
    image[np.logical_not(mask)] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0,max=100,step=1,value=60,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0,max=num_frames,step=1,value=25,
                             layout=Layout(width='70%'))

interact(update,w=slider_w, i=slider_i)

In [ ]:
print(recording)

In [ ]:
threshold = 0.59
masks = np.zeros(flow_recording.shape[:3],dtype=bool)

if recording == "stack_recordings/episode_118":
    switch_frame = 65
elif recording == "stack_recordings/episode_129":
    switch_frame = 65
elif recording == "stack_recordings/episode_156":
    switch_frame = 80
elif recording == "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/stacking":
    switch_frame = 105
else:
    switch_frame = 94
    
for i in range(len(flow_recording)):
    if i < switch_frame:
        color_index = 2
    else:
        color_index = 0
        threshold = .76
    mask = get_mask(flow_recording[i], threshold, color_index, i)
    masks[i] = mask

print(np.mean(masks) * 100, "percent of pixels fg")
if folder_format == 'max':
    mask_fn = flow_recording_fn.replace("_img.npz","_mask.npz")
else:
    mask_fn = flow_recording_fn.replace(".npz","_mask.npz")
np.savez(mask_fn, mask=masks)
print("Saved to",mask_fn)

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(masks[25])
ax.set_axis_off()

def update(i):
    image = flow_recording[i].copy()
    mask = masks[i]
    image[np.logical_not(mask)] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=num_frames,step=1,value=25,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)

In [ ]:
# This should select a bounding box.

index_A = 25
index_B = 5

fig, ((ax, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax.imshow(flow_recording[index_A])
ax.set_axis_off()
tmp = flow_recording[index_A].copy()
tmp[20:60,35:55] = 255

ax2.imshow(tmp)
ax2.set_axis_off()

ax3.imshow(flow_recording[index_B])



# Masking based on Depth

In [ ]:
episode_data = np.load(state_recording_fn)
keys = list(episode_data.keys())
print(keys)

depth = episode_data["depth_imgs"]
depth_scale = 8000


x = np.linspace(0, 2 * np.pi)
fig, ax = plt.subplots(1)
line = ax.imshow(flow_recording[0])
ax.set_axis_off()

depth_min, depth_max = int(depth.min()*depth_scale), int(depth.max()*depth_scale)

def get_mask_depth(frame, l, h):
    mask = np.logical_or(frame < l/depth_scale, frame > h/depth_scale)
    mask = np.logical_not(mask)
    return mask
    
def update(w,l,h):
    vline.set_data([w,w],[0,1])
    mask = get_mask_depth(depth[w], l, h)
    mask = np.logical_not(mask)
    display_image = flow_recording[w].copy()
    display_image[mask] = 0
    line.set_data(display_image)
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0,max=num_frames,step=1,value=205,
                             layout=Layout(width='70%'))

slider_l = widgets.IntSlider(min=depth_min,max=depth_max,step=1,value=1650,
                             layout=Layout(width='70%'))
slider_h = widgets.IntSlider(min=depth_min,max=depth_max,step=1,value=2250,
                             layout=Layout(width='70%'))

interact(update,w=slider_w,l=slider_l,h=slider_h)


In [ ]:
threshold_low = slider_l.value
threshold_high = slider_h.value

masks = np.zeros(flow_recording.shape[:3],dtype=bool)

    
for i in range(len(flow_recording)):
    mask = get_mask_depth(depth[i], threshold_low, threshold_high)
    masks[i] = mask

print(np.mean(masks) * 100, "percent of pixels fg")
mask_fn = flow_recording_fn.replace(".npz","_mask.npz")
np.savez(mask_fn, mask=masks)
print("Saved to",mask_fn)

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(masks[25])
ax.set_axis_off()

def update(i):
    image = flow_recording[i].copy()
    mask = masks[i]
    image[mask] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=num_frames,step=1,value=200,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)